#Import libraries and mount drive

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingRegressor


import xgboost as xgb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.impute import SimpleImputer

from sklearn import svm
from sklearn.svm import SVR
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data pre-processing and feature engineering

In [ ]:
players_21 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_21.csv')


In [ ]:
players_22 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_22.csv')

<ipython-input-3-c0af8ff2dfb1>:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/colab files/players_22.csv')


In [ ]:
players_21.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18944 entries, 0 to 18943
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 15.9+ MB


In [ ]:
columns_list = players_21.columns.tolist()
print(columns_list)

['sofifa_id', 'player_url', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id', 'club_name', 'league_name', 'league_level', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power

In [ ]:
#Dropping columns with data sets that have 50% missing values
cols_to_drop = []
for i in players_21.columns:
    missing = np.abs((players_21[i].count() - players_21[i].shape[0])/players_21[i].shape[0] * 100)
    if missing > 50:
        print('{} - {}%'.format(i, round(missing)))
        cols_to_drop.append(i)

players_21.drop(columns=cols_to_drop,inplace=True)
print(players_21.shape)

columns_to_drop = ['sofifa_id', 'player_url', 'player_face_url', 'club_logo_url', 'club_flag_url', 'nation_flag_url']
players_21.drop(columns=columns_to_drop,inplace=True)

In [ ]:
# Split dataset into categorical and numeric
selected_columns= ['club_name','club_position','league_name','preferred_foot']
categorical_data =players_21[selected_columns]
numeric_data = players_21.select_dtypes(exclude=['object'])


#imputing numeric
imp=SimpleImputer()
imputed_data=imp.fit_transform(numeric_data)
df_numeric =pd.DataFrame(imputed_data, columns=numeric_data.columns)

#imputing non_numeric
df = pd.DataFrame(categorical_data )
columns=categorical_data.columns
modes = df[columns].mode().iloc[0]
df[columns] = df[columns].fillna(modes)


In [ ]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

columns_to_encode = df.columns  # Replace with your actual column names

encoded_df = pd.DataFrame()
df['preferred_foot'] = label_encoder.fit_transform(df['preferred_foot'])
df['club_position'] =  label_encoder.fit_transform(df['club_position'])
df['club_name'] =  label_encoder.fit_transform(df['club_name'])
df['league_name'] =  label_encoder.fit_transform(df['league_name'])

df

In [ ]:
New_players_21 = pd.concat([df_numeric,df], axis=1)
New_players_21

In [ ]:
# Replace these with your data and model
overall= New_players_21['overall']
features= New_players_21.drop(columns='overall')
X = pd.DataFrame(features)  # Your feature data
y = pd.Series(overall)    # Your target variable
rf_model = RandomForestRegressor()  # Your trained model

# Train the model on your data
rf_model.fit(X, y)

importances = rf_model.feature_importances_

# Create a DataFrame to visualize feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
# Print or visualize feature importance
feature_importance_df

In [ ]:
features= New_players_21.drop(columns='overall')# Replace with your actual feature names
X = pd.DataFrame(features)
your_feature_names= X.columns

# Sort features by importance
feature_indices = np.argsort(importances)[::-1]

# Select the top 10 features
top_10_feature_indices = feature_indices[:18]

# Get the names of the top 10 features
top_15_feature_names = [your_feature_names[i] for i in top_10_feature_indices]
top_15_feature_names

['value_eur',
 'release_clause_eur',
 'age',
 'potential',
 'movement_reactions',
 'wage_eur',
 'defending',
 'club_name',
 'mentality_interceptions',
 'league_name',
 'attacking_crossing',
 'goalkeeping_diving',
 'mentality_composure',
 'goalkeeping_reflexes',
 'defending_marking_awareness',
 'goalkeeping_positioning',
 'defending_standing_tackle',
 'mentality_penalties']

#Checking for maximum correlation


In [ ]:
cols= ['value_eur',
 'release_clause_eur',
 'age','overall',
 'potential',
 'movement_reactions',
 'wage_eur',
 'defending',
 'club_name',
 'mentality_interceptions',
 'league_name',
 'attacking_crossing',
 'goalkeeping_diving',
 'mentality_composure',
 'goalkeeping_reflexes',
 'defending_marking_awareness',
 'goalkeeping_positioning',
 'defending_standing_tackle',
 'mentality_penalties']
Cleaned_player_21= New_players_21[cols ]

In [ ]:
def find_max_corr_features(data, dependent_variable, num_subsets):
    max_corr_subsets = []

    # Correlation matrix
    corr_matrix = data.corr().abs()

    # Column indices of features sorted by correlation with dependent variable
    sorted_features = np.argsort(corr_matrix[dependent_variable])[::-1]

    for _ in range(num_subsets):
        subset = []
        for i in sorted_features:
            if i not in subset:
                subset.append(i)
                if len(subset) == data.shape[1] - 1:
                    break
        max_corr_subsets.append(subset)

    return max_corr_subsets


dependent_variable = 'overall'
num_subsets = 15

max_corr_subsets = find_max_corr_features(Cleaned_player_21, dependent_variable, num_subsets)

for i, subset in enumerate(max_corr_subsets):
    print(f"Subset {i + 1}: {subset}")

Subset 1: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 2: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 3: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 4: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 5: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 6: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 7: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 8: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 9: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 10: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 11: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 12: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 13: [3, 5, 13, 4, 1, 6, 0, 2, 11, 7, 18, 15, 9, 17, 16, 10, 14, 12]
Subset 14: [3, 5, 13, 4, 1, 6, 0, 

In [ ]:
X = Cleaned_player_21.drop(columns='overall')
y = players_21['overall']

#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

from joblib import dump
dump(scaler, 'scaler.joblib')

Training suitable machine learning models with cross-validation

#The Random forest Model

In [ ]:
# Initialize the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation and print the mean accuracy score
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5)
mean_cv_scores_rf=np.mean(cv_scores)
print(f"Random Forest Model CV Mean Accuracy Score: {mean_cv_scores_rf}")

Random Forest Model CV Mean Accuracy Score: 0.9949660328597286


In [ ]:
#Make the prediction
y_pred = cross_val_predict(rf_model, X_train, y_train, cv=5)
print(rf_model)
#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

RandomForestRegressor(random_state=42)
MAE: 0.28228373474100976
RMSE: 0.49902644248599787


In [228]:
#Fine tuning the random forest model
# Define the parameter grid
param_grid_rf = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 3, 5],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}

# Create a RandomForestRegressor model
rf_model = RandomForestRegressor(random_state=42)

# Create the GridSearchCV object
grid_search_rf = GridSearchCV(
    rf_model, param_grid_rf, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1
)

# Fit the model to the training data
grid_search_rf.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 5],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='neg_mean_absolute_error')

In [230]:
# Get the best parameters and best estimator
best_params = grid_search_rf.best_params_
best_rf_model = grid_search_rf.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
y_pred = best_rf_model.predict(X_test)

#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_test, y_pred )
rmse = np.sqrt(mean_squared_error(y_test, y_pred ))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
MAE: 0.2524795460543679
RMSE: 0.458368270459983


#The Gradient Boosting Model

In [242]:
# Initialize the model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# Perform cross-validation and print the mean accuracy score
cv_scores = cross_val_score(gb_model, X_train, y_train, cv=5)
mean_cv_scores_gr = np.mean(cv_scores)
print(f"Gradient Boosting Model CV Mean Accuracy Score: {mean_cv_scores_gr}")


Gradient Boosting Model CV Mean Accuracy Score: 0.987042481600052


In [243]:
#Make the prediction
y_pred = cross_val_predict(gb_model, X_train, y_train, cv=5)
print(y_pred)
#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

[49.79391239 55.36701472 67.55387371 ... 70.24190653 77.3986622
 59.18018458]
MAE: 0.5866570858825924
RMSE: 0.8004806693204713


In [252]:
# fine tuning the gradient boost

# Define the parameter grid
param_grid_gb = {
    'n_estimators': [50,150],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 5],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'subsample': [0.8, 0.9]
}

# Create a GradientBoostingRegressor model
gb_model = GradientBoostingRegressor()

# Create the GridSearchCV object
grid_search_gb = GridSearchCV(
    gb_model, param_grid_gb, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1
)

# Fit the model to the training data

In [253]:
grid_search_gb.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05], 'max_depth': [4, 5],
                         'min_samples_leaf': [1], 'min_samples_split': [2],
                         'n_estimators': [50, 150], 'subsample': [0.8, 0.9]},
             scoring='neg_mean_absolute_error')

In [254]:
#Measure the model's performance using the Mean absolute error and the Root mean squared error
# Get the best parameters and best estimator
best_params = grid_search_gb.best_params_
best_gb_model = grid_search_gb.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
y_pred = best_gb_model.predict(X_test)

# You can use metrics like mean absolute error, mean squared error, etc. to evaluate the model.
mae = mean_absolute_error(y_test, y_pred )
rmse = np.sqrt(mean_squared_error(y_test, y_pred ))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

Best Parameters: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150, 'subsample': 0.8}
MAE: 0.3815831892026116
RMSE: 0.5461495666785222


# The XGBoost model

In [235]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100)

xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

average_score = np.mean(xgb_scores)

print("Average 5-Fold Cross-Validation Score: {:.4f}".format(average_score))

Average 5-Fold Cross-Validation Score: -0.2646


In [236]:
xgb_predictions = cross_val_predict(xgb_model, X_train, y_train, cv=5)
print(xgb_predictions)

#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_train, xgb_predictions)
rmse = np.sqrt(mean_squared_error(y_train, xgb_predictions))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

[49.847084 54.623714 67.040054 ... 69.9997   77.64766  58.761818]
MAE: 0.3535913737368481
RMSE: 0.5144169828917263


In [238]:
#fine tuning the xbgboost

# Define the parameter grid
param_grid_xgb = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
}

# Create an XGBRegressor model
xgb_model = XGBRegressor(random_state=42)

# Create the GridSearchCV object
grid_search_xgb = GridSearchCV(
    xgb_model, param_grid_xgb, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1
)

# Fit the model to the training data
grid_search_xgb.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search_xgb.best_params_
best_xgb_model = grid_search_xgb.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)

# Evaluate the best model on the test data
y_pred = best_xgb_model.predict(X_test)

#Measure the model's performance using the Mean absolute error and the Root mean squared error
mae = mean_absolute_error(y_test, y_pred )
rmse = np.sqrt(mean_squared_error(y_test, y_pred ))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

Best Parameters: {'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 2, 'n_estimators': 200, 'subsample': 1.0}
MAE: 0.32042474613305477
RMSE: 0.4644767454362285


#Custom ensemble model: The stacking regressor

Determine the ensemble's model architecture

In [255]:
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Create the base regressor models
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
rf_model = RandomForestRegressor(random_state=42)
gb_model = GradientBoostingRegressor(random_state=42)

# Create the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('rf', rf_model),
        ('gb', gb_model)
    ],
    final_estimator=XGBRegressor(random_state=42)  # You can adjust hyperparameters here
)

# Fit the stacking regressor to the training set
stacking_regressor.fit(X_train, y_train)

# Predict the target variable for the test set
stacking_preds = stacking_regressor.predict(X_test)

mae = mean_absolute_error(y_test, stacking_preds)
rmse = np.sqrt(mean_squared_error(y_test, stacking_preds))

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

best_model= stacking_regressor


MAE: 0.24585374523574616
RMSE: 0.47100813937535624


#Test ensemble model on the Players_22 data

Load the trained ensemble model and test it's performance on the Players_22 data

In [270]:
# Test the model on players_22
cols= ['value_eur',
 'release_clause_eur',
 'age','overall',
 'potential',
 'movement_reactions',
 'wage_eur',
 'defending',
 'mentality_interceptions',
 'attacking_crossing',
 'goalkeeping_diving',
 'mentality_composure',
 'goalkeeping_reflexes',
 'defending_marking_awareness',
 'goalkeeping_positioning',
 'defending_standing_tackle',
 'mentality_penalties','club_name','league_name']
players_22_Data= players_22[cols]

In [271]:
# Split dataset into categorical and numeric
selected_columns= ['club_name','league_name']
categorical_data =players_22_Data[selected_columns]
numeric_data = players_22_Data.select_dtypes(exclude=['object'])


#imputing numeric
imp=SimpleImputer()
imputed_data=imp.fit_transform(numeric_data)
df_numeric =pd.DataFrame(imputed_data, columns=numeric_data.columns)

#imputing non_numeric
df = pd.DataFrame(categorical_data )
columns=categorical_data.columns
modes = df[columns].mode().iloc[0]
df[columns] = df[columns].fillna(modes)

In [272]:
label_encoder = LabelEncoder()

columns_to_encode = df.columns  # Replace with your actual column names

encoded_df = pd.DataFrame()

df['club_name'] =  label_encoder.fit_transform(df['club_name'])
df['league_name'] =  label_encoder.fit_transform(df['league_name'])

New_players_22 = pd.concat([df_numeric,df], axis=1)
New_players_22.columns

Index(['value_eur', 'release_clause_eur', 'age', 'overall', 'potential',
       'movement_reactions', 'wage_eur', 'defending',
       'mentality_interceptions', 'attacking_crossing', 'goalkeeping_diving',
       'mentality_composure', 'goalkeeping_reflexes',
       'defending_marking_awareness', 'goalkeeping_positioning',
       'defending_standing_tackle', 'mentality_penalties', 'club_name',
       'league_name'],
      dtype='object')

In [275]:
X = New_players_22.drop(columns='overall')
X.columns

Index(['value_eur', 'release_clause_eur', 'age', 'potential',
       'movement_reactions', 'wage_eur', 'defending',
       'mentality_interceptions', 'attacking_crossing', 'goalkeeping_diving',
       'mentality_composure', 'goalkeeping_reflexes',
       'defending_marking_awareness', 'goalkeeping_positioning',
       'defending_standing_tackle', 'mentality_penalties', 'club_name',
       'league_name'],
      dtype='object')

In [277]:
X = New_players_22.drop(columns='overall')
y = New_players_22['overall']
X = X[['value_eur', 'release_clause_eur', 'age', 'potential', 'movement_reactions', 'wage_eur', 'defending', 'club_name','mentality_interceptions','league_name', 'attacking_crossing', 'goalkeeping_diving', 'mentality_composure', 'goalkeeping_reflexes', 'defending_marking_awareness', 'goalkeeping_positioning', 'defending_standing_tackle', 'mentality_penalties']]

#Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X = scaler.fit_transform(X)

y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.6691602158301742


In [269]:
import pickle
# Use pickle to save the trained model
with open('best_model.plk', 'wb') as f:
    pickle.dump(best_model, f)

